In [1]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np
config = OmegaConf.load("./configs/default_config_linearprobe50.yaml")

for dataset in [ 'uc-merced-land-use-dataset' ]:
                # 'kvasir-dataset'
                # 'stanford_cars',
                #'caltech101', 'dtd', 'eurosat' ,
                # 'fgvc-aircraft-2013b', 'flowers-102', 'oxford-iiit-pet'
                #'Imagenette' ]:
    
    config.dataset = dataset
    train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(False, config) 

    print(dataset, len(train_dataset), len(val_dataset), len(test_dataset),  N)

/home/mheuillet/Desktop/robust_training/.venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-07 18:01:52,071	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-07 18:01:52,167	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-07 18:01:52,251	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


uc-merced-land-use-dataset 1428 252 420 21


In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Path to your unzipped Kvasir dataset
dataset_root = '/home/mheuillet/Desktop/robust_training/data/kvasir_dataset_unzipped/kvasir-dataset'

# Define any transforms you need (e.g., resizing, normalization, etc.)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),   # or some other size
    transforms.ToTensor(),
    # Optional: transforms.Normalize(mean, std),
])

# Create an ImageFolder dataset
kvasir_dataset = datasets.ImageFolder( root=dataset_root, transform=data_transforms )

# Check the classes that ImageFolder identified
print("Classes found:", kvasir_dataset.classes)
# e.g. ["dyed-lifted-polyps", "dyed-resection-margins", ...]
# Each subfolder is assigned a class index in alphabetical order

# Create a DataLoader for batching
dataloader = DataLoader(kvasir_dataset, batch_size=32, shuffle=True, num_workers=4)

# Iterate over a few batches to verify it works
for images, labels in dataloader:
    print("Batch images shape:", images.shape)
    print("Batch labels shape:", labels.shape)
    break  # Just show the first batch, then stop




Classes found: ['dyed-lifted-polyps', 'dyed-resection-margins', 'esophagitis', 'normal-cecum', 'normal-pylorus', 'normal-z-line', 'polyps', 'ulcerative-colitis']
Batch images shape: torch.Size([32, 3, 224, 224])
Batch labels shape: torch.Size([32])


In [ ]:
### don't pay attention to this cell

# EuroSAT: seulement 8 runs on complete et tous les autres OOM
# Caltech: 26 runs ok, 
# Stanford Cars: 34 ok
# Aircraft: 26 ok
# DTD : 26 ok
# Oxford pet: 26 ok
# Flowers: 26 ok

stanford_cars = 6922 + 1222 + 8041 
caltech101 = 5899 + 1042 + 1736 
dtd = 1880 + 1880 + 1880 
eurosat = 18360 + 3240 + 5400 
fgvcaircraft2013b =  3334 + 3333 + 3333 
flowers102 = 1020 + 1020 + 6149 
oxfordiiitpet = 3128 + 552 + 3669 

print(eurosat, caltech101, dtd, fgvcaircraft2013b, flowers102, oxfordiiitpet, stanford_cars)

from PIL import Image
import torchvision.datasets as datasets

# Initialize the Caltech101 dataset without transformations
caltech101_sample = datasets.Caltech101(root='./data', download=False)
print(len(caltech101_sample))
for i in range( len(caltech101_sample) ) :
    image, label = caltech101_sample[i]

    # Check the image mode
    # print(f"Image Mode: {image.mode}")  # Expected: 'RGB'
    if image.mode != 'RGB':
        print('issue')